In [1]:
import requests
from datetime import datetime
from src.utils import loadConfig
import pandas as pd
import pyodbc
import os

In [2]:
def api_call(config, location='Australia', timesteps='1d', units='metric'):
    API_KEY = config['API_KEY']
    url = f"https://api.tomorrow.io/v4/weather/forecast?location={location.lower()}&timesteps={timesteps}&units={units}&apikey={API_KEY}"
    headers = {"accept": "application/json"}
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
        
    except requests.RequestException as e:
        raise ValueError(f"Error fetching data from the API: {e}") from None

def response_to_df(response):
    dates = []
    hours = []
    weather_values_list = []

    for data in response['timelines']['hourly']:
        date = datetime.fromisoformat(data['time'].replace('Z', '+00:00'))
        hours.append(date.hour)
        dates.append(date.date())
        
        weather_values = data['values']
        weather_values_list.append(weather_values)

    df = pd.DataFrame(weather_values_list)
    df.insert(0, 'Date', dates)
    df.insert(1, 'Hour', hours)
    
    features = ['Date', 'Hour', 'temperature','precipitationProbability','windSpeed']
    df = df[features].rename(columns={'precipitationProbability':'RainProb',
                                      'temperature':'Temperature',
                                      'windSpeed':'WindSpeed'})
    df['WindSpeed'] = df['WindSpeed']*3.6

    temp_threshold = 35
    windspeed_threshold = 45
    rainprob_threshold = 50

    df.loc[:, 'HeavyWeather'] = (
        (df['Temperature'] > temp_threshold) |
        (df['RainProb'] > rainprob_threshold) |
        (df['WindSpeed'] > windspeed_threshold)
    ).astype(int)
    
    return df

def df_to_sqlserver(df, config):
    server = config["SERVER"]
    database = config["DATABASE"]
    table_name = config["WEATHER_TABLE"]

    conn_str = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'

    try:
        conn = pyodbc.connect(conn_str)
        cursor = conn.cursor()
        
        cursor.execute(f"SELECT COUNT(*) FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = ?", (table_name,))
        table_exists = cursor.fetchone()[0] > 0

        if not table_exists:
            raise ValueError(f"Table '{table_name}' does not exist. Aborting the operation.")
        
        for _, data in df.iterrows():
            cursor.execute(f"SELECT COUNT(*) FROM {table_name} WHERE Date = ? AND Hour = ?", (data['Date'],data['Hour']))
            date_exists = cursor.fetchone()[0] > 0
            
            if date_exists:
                query = f"""
                    UPDATE {table_name}
                    SET Temperature = ?,
                        RainProb = ?,
                        WindSpeed = ?,
                        HeavyWeather = ?
                    WHERE Date = ? and Hour = ?
                """
                cursor.execute(query, (data['Temperature'], data['RainProb'], data['WindSpeed'], data['HeavyWeather'], data['Date'], data['Hour']))
            else:
                query = f"""
                    INSERT INTO {table_name} (Date, Hour, Temperature, RainProb, WindSpeed, HeavyWeather)
                    VALUES (?, ?, ?, ?, ?, ?)
                """
                cursor.execute(query, (data['Date'], data['Hour'], data['Temperature'], data['RainProb'], data['WindSpeed'], data['HeavyWeather']))
            
            conn.commit()
        
        print("All weather data successfully moved to the database.")
        
        conn.close()

    except pyodbc.Error as e:
        print("Error connecting to SQL Server:", e)

def read_table(config, save=False):
    try:
        server = config["SERVER"]
        database = config["DATABASE"]
        conn_str = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'
        conn = pyodbc.connect(conn_str)

        query = "SELECT * FROM Weather"

        df = pd.read_sql(query, conn)
        if save :
            path = config["WEATHER_PATH"]
            df.to_csv(path,index=False) 

        conn.close()

        return df
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None
    

In [3]:
config = loadConfig('config.yaml')
response = api_call(config, 'Australia','1h','metric')
df = response_to_df(response)
df_to_sqlserver(df, config)
read_table(config, save=True)

All weather data successfully moved to the database.


C:\Users\Acer\AppData\Local\Temp\ipykernel_5444\3445540736.py:105: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,Date,Hour,Temperature,RainProb,WindSpeed,HeavyWeather
0,2024-05-02,8.0,22.63,0.0,19.116,0
1,2024-05-02,9.0,20.96,0.0,14.040,0
2,2024-05-02,10.0,19.49,0.0,13.212,0
3,2024-05-02,11.0,18.29,0.0,13.140,0
4,2024-05-02,12.0,17.17,0.0,14.436,0
...,...,...,...,...,...,...
426,2024-05-21,0.0,13.97,0.0,23.328,0
427,2024-05-21,1.0,15.51,0.0,24.660,0
428,2024-05-21,2.0,17.04,0.0,25.956,0
429,2024-05-21,3.0,18.57,0.0,27.288,0
